# 7. Visualização Cartográfica
---

_“A criação de mapas é um dos empreendimentos intelectuais mais antigos da humanidade e também um dos mais complexos, com teoria científica, representação gráfica, fatos geográficos e considerações práticas combinados de maneiras infinitas.”_ &mdash; [H. J. Steward](https://books.google.com/books?id=cVy1Ms43fFYC)

Cartografia &ndash; o estudo e a prática da criação de mapas &ndash; tem uma história rica que abrange séculos de descobertas e design. A visualização cartográfica aproveita técnicas de mapeamento para transmitir dados contendo informações espaciais, como localizações, rotas ou trajetórias na superfície da Terra.

<div style="float: right; margin-left: 1em; margin-top: 1em;"><img width="300px" src="https://gist.githubusercontent.com/jheer/c90d582ef5322582cf4960ec7689f6f6/raw/8dc92382a837ccc34c076f4ce7dd864e7893324a/latlon.png" /></div>

Aproximando a Terra como uma esfera, podemos denotar posições usando um sistema de coordenadas esféricas de _latitude_ (ângulo em graus ao norte ou ao sul do _equador_) e _longitude_ (ângulo em graus especificando a posição leste-oeste). Neste sistema, um _paralelo_ é um círculo de latitude constante e um _meridiano_ é um círculo de longitude constante. O [_meridiano principal_](https://en.wikipedia.org/wiki/Prime_meridian) está em 0° de longitude e, por convenção, é definido para passar pelo Royal Observatory em Greenwich, Inglaterra.

Para "achatar" uma esfera tridimensional em um plano bidimensional, devemos aplicar uma [projeção](https://en.wikipedia.org/wiki/Map_projection) que mapeia pares (`longitude`, `latitude`) para coordenadas (`x`, `y`). Semelhante às [escalas](https://github.com/uwdata/visualization-curriculum/blob/master/altair_scales_axes_legends.ipynb), as projeções mapeiam de um domínio de dados (posição espacial) para um intervalo visual (posição em pixels). No entanto, os mapeamentos de escala que vimos até agora aceitam um domínio unidimensional, enquanto as projeções de mapas são inerentemente bidimensionais.

Neste _notebook_, apresentaremos o básico sobre a criação de mapas e visualização de dados espaciais com Altair, incluindo:

- Formatos de dados para representar características geográficas,
- Técnicas de geo-visualização, como mapas de pontos, símbolos e coropléticos, e
- Uma revisão das projeções cartográficas mais comuns.

_Este notebook faz parte do [currículo de visualização de dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt
from vega_datasets import data

## 7.1 Dados Geográficos: GeoJSON e TopoJSON
---

Até esse ponto, trabalhamos com datasets formatados em JSON e CSV, que correspondem a tabelas de dados feitas de linhas (registros) e colunas (campos). Com o objetivo de representar regiões geográficas (países, estados, _etc._) e trajetórias (rotas de voo, linhas de metrô, _etc._), precisamos  expandir nosso repertório com formatos adicionais projetados para suportar geometrias ricas.

O [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) modela recursos geográficos com umformato JSON especializado. Um `feature` GeoJSON pode incluir dados geométricos – tal como coordenadas de `longitude` e `latitude`, que constituem a fronteira de um país – assim como atributos de dados adicionais.

Aqui está um objeto `feature` GeoJSON para o Colorado, um estado americano:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

O `feature` inclui um objeto `properties`, que pode incluir qualquer número de campos de dados, mais um objeto `geometry`, que nesse caso contém um único polígono que consiste de coordenadas `[longitude, latitude]` para a fronteira do estado. As coordenadas continuam para a direita por um tempo, caso você queira rolar...

Para aprender  mais sobre os detalhes minuciosos do GeoJSOn, veja as [especificações oficiais de GeoJSON](https://geojson.org/) ou leia a [útil cartilha de Tom MacWright.](https://macwright.com/2015/03/23/geojson-second-bite)

Uma desvantagem do GeoJSON como um formato de armazenamento é que ele pode ser redundante, resultando um arquivos maiores. Considere: O Colorado faz fronteira com outros seis estados (sete se você incluir o canto tocando o Arizona). Ao invés de usar listas de coordenadas separadas e sobrepostas para cada um desses estados, uma abordagem mais compacta é codificar fronteiras compartilhadas somente uma vez, representando a _topologia_ das regiões geográficas. Felizmente, isso é exatamente o que o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz!

vamos carregar um arquivo TopoJSON de países do mundo (com resolução de 110 metros):

In [2]:
world = data.world_110m.url
world

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'

In [3]:
world_topo = data.world_110m()

In [4]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [5]:
world_topo['type']

'Topology'

In [6]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_Inspecione o dicionário TopoJSON `world_topo` acima paraver seu conteúdo._

Nos dados acima, a propriedade `objects` indica os elementos nomeados que podemos extrair dos dados: geometrias para todos os países (`countries`), ou um único polígono representando todas terras (`land`) do planeta Terra. Qualquer um deles pode ser descompactado para dados GeoJSON que poderemos visualizar.

Como TopoJSON é um formato especializado, precisamos instruir o Altair a analisar o formato TopoJSON, indicando qual objeto de recurso nomeado desejamos extrair da topologia. O código seguinte indica que queremos extrair recursos GeoJSON do dataset `world` para o objeto `countries`:


In [7]:
alt.topo_feature(world, 'countries')

UrlData({
  format: TopoDataFormat({
    feature: 'countries',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'
})

Essa chamada de método `alt.topo_feature` se expande para o seguinte JSON do Vega-Lite:

In [8]:
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}

{'values': 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json',
 'format': {'type': 'topojson', 'feature': 'countries'}}

Agora que podemos carregar dados geográficos, estamos prontos para começar a fazer mapas!

## 7.2 Marcas de Geoforma
---

Para visualizar dados geográficos, o Altair fornece o tipo de marca `geoshape`. Para criar um mapa básico, podemos criar uma marca `geoshape` e passar nossos dados TopoJSON, que são então desembrulhados em recursos GeoJSON, um para cada país do mundo:


In [12]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplica uma cor azul padrão e usa uma projeção de mapa padrão (`mercator`). Podemos personalizar as cores e as larguras das bordas usando propriedades padrão de marca. Usando o método `project`, também podemos adicionar nossa própria projeção de mapa:


In [13]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados caibam dentro da largura e altura do gráfico. Também podemos especificar parâmetros de projeção, como `scale` (nível de zoom) e `translate` (panorama), para personalizar as configurações da projeção. Aqui, ajustamos os parâmetros `scale` e `translate` para focar na Europa:


In [14]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Observe como a resolução de 110m dos dados se torna aparente neste nível de escala. Para ver linhas de costa e fronteiras mais detalhadas, precisamos de um arquivo de entrada com geometrias mais detalhadas. Ajuste os parâmetros `scale` e `translate` para focar o mapa em outras regiões!_


Até agora, nosso mapa mostra apenas países. Usando o operador `layer`, podemos combinar múltiplos elementos do mapa. O Altair inclui _geradores de dados_ que podemos usar para criar dados para camadas adicionais do mapa:

- O gerador de esfera (`{'sphere': True}`) fornece uma representação GeoJSON da esfera completa da Terra. Podemos criar uma marca `geoshape` adicional que preenche a forma da Terra como uma camada de fundo.
- O gerador de graticule (`{'graticule': ...}`) cria um recurso GeoJSON representando um _graticule_: uma grade formada por linhas de latitude e longitude. O graticule padrão tem meridianos e paralelos a cada 10° entre ±80° de latitude. Para as regiões polares, há meridianos a cada 90°. Essas configurações podem ser personalizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos sobrepor as marcas de esfera, graticule e país em uma especificação de mapa reutilizável:


In [15]:
map = alt.layer(
    # use a esfera da Terra como camada base
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # adicione um graticule para as linhas de referência geográficas
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # e depois os países do mundo
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)


Podemos estender o mapa com a projeção desejada e desenhar o resultado. Aqui aplicamos uma [projeção Natural Earth](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ fornece o fundo azul claro; a camada _graticule_ fornece as linhas de referência geográficas brancas.


In [16]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)

## 7.3 Mapas de Pontos
---

Em adição aos dados _geometric_ fornecidos pelo GeoJSON ou TopoJSON, muitos datasets tabulares incluem informação geográfica na forma das coordenadas nos campos `longitude` e `latitude`, ou referencia regiões geográficas como nomes de países, estados, códigos postais, _etc_, que podem ser mapeados para coordenadas usando [Address geocoding](https://en.wikipedia.org/wiki/Address_geocoding). Em alguns casos, os dados de localização são ricos o suficientes para ver padrões significativos ao projetar os pontos sozinhos - Nenhum mapa base é necessário!

Vamos dar uma olhada no dataset de códigos postais de 5 dígitos dos EUA, incluindo as coordenadas `longitude`, `latitude` para cada agência de correios em adição ao campo `.longitudelatitudezip_code`

In [17]:
zipcodes = data.zipcodes.url
zipcodes

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/zipcodes.csv'

In [20]:
alt.Chart(zipcodes).mark_square(
    size=1, opacity=1
).encode(
    longitude='longitude:Q', # aplica o campo 'longitude' ao canal de longitude
    latitude='latitude:Q'    # aplica o campo 'latitude' ao canal de latitude
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

Nós podemos visualizar a localização de cada agência de correios usando uma pequena ( 1-pixel ) marcação. Entretanto, para definir as posições nós _não_ usamos os canais `x` e `y`. _Mas por que não?_

Enquanto as projeções cartográficas mapeiam coordenadas (`longitude`, `latitude`) em coordenadas (`x`, `y`), eles podem fazer isso de formas arbitrárias. Não há garantia, por exemplo, de que  `longitude` → `x` e `latitude` → `y`! Em vez disso, o altair inclui os canais de codificação especiais `latitude`e `longitude` para lidar com coordenadas geográficas. Esses canais indicam quais campos de dados devem ser mapeados para as coordenadas de `longitude` e `latitude` e então aplica uma projeção para mapear essas coordenadas para as posições  `x` e `y`.

_Plotando apenas códigos postais, podemos ver o contorno dos Estados Unidos e discernir padrões significativos na densidade de agências de correio, sem um mapa base ou elementos de referência adicionais!_

Usamos a projeção `albersUsa`, que toma algumas liberdades com a geometria real da Terra, com versões em escala do Alasca e Havaí no canto inferior esquerdo. Como não especificamos os parâmetros `scale` ou `translate` da projeção, o Altair os define automaticamente para se ajustarem aos dados visualizados.

Agora podemos fazer mais perguntas sobre nosso conjunto de dados. Por exemplo, há algum padrão ou razão para a alocação de códigos postais? Para reponder essa questão, podemos adicionar uma codificação de cores baseado no primeiro dígito do código postal. Primeiro, adicionamos uma transformação `calculate` para extrair o primeiro dígito e codificamos o resultado usando o canal de cores:

In [21]:
alt.Chart(zipcodes).transform_calculate(
    digit='datum.zip_code[0]'
).mark_square(
    size=2, opacity=1
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Para observar um dígito específico, adicionamos uma filtragem para para limitar os dados exibidos! Tente adicionar uma [seleção interativa](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) para filtrar para um único dígito e atualizar dinamicamente o mapa. E certifique-se de usar strings (\`'1'\`) em vez de números (\`1\`) ao filtrar valores de dígitos!_

(Este exemplo é inspirado na visualização clássica [zipdecode](https://benfry.com/zipdecode/) de Ben Fry!)

Podemos nos perguntar ainda mais o que a _sequência_ de códigos postais pode indicar. Uma maneira de explorar essa questão é conectar cada código postal consecutivo usando uma marca de `linha`, como feito na visualização [ZipScribble](https://eagereyes.org/zipscribble-maps/united-states) de Robert Kosara:

In [22]:
alt.Chart(zipcodes).transform_filter(
    '-150 < datum.longitude && 22 < datum.latitude && datum.latitude < 55'
).transform_calculate(
    digit='datum.zip_code[0]'
).mark_line(
    strokeWidth=0.5
).encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    color='digit:N',
    order='zip_code:O'
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

_Agora podemos ver como os códigos postais se agrupam ainda mais em áreas menores, indicando uma alocação hierárquica de códigos por localização, mas com alguma variabilidade notável dentro dos clusters locais._

Se você estava prestando atenção aos nossos mapas anteriores, pode ter notado que há códigos postais sendo plotados no canto superior esquerdo! Eles correspondem a locais como Puerto Rico ou American Samoa, que contêm códigos postais dos EUA, mas são mapeados para coordenadas `nulas` (`0`, `0`) pela projeção `albersUsa`. Além disso, Alasca e Havaí podem complicar nossa visão dos segmentos de linha de conexão. Em resposta, o código acima inclui um filtro adicional que remove pontos fora dos nossos intervalos de `longitude` e `latitude` escolhidos.

_Remova o filtro acima para ver o que acontece!_

## 7.4 Mapas de Símbolos
---

Agora, vamos combinar um mapa base com os dados visualizados como camadas diferentes. Examinaremos o networking dos voôs comerciais nos Estados Unidos, considerando tanto os aeroportos como as rotas dos voôs. Para fazermos isso, precisaremos de três conjuntos de dados diferentes. Para nosso mapa base, usaremos um arquivo do tipo TopoJSON para os Estados Unidos numa resolução de 10m, contendo dados para estados (`states`) e municípios (`counties`):

In [23]:
usa = data.us_10m.url
usa

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/us-10m.json'

Para os aeroportos, vmos usar a base de dados com os campos das coordenadas de `latitude` e `longitude` de cada aeroporto, bem como o código `iata` do aeroporto — por exemplo, `SEA` para o [Aeroporto Internacional de Seattle-Tacoma](https://pt.wikipedia.org/wiki/Aeroporto_Internacional_de_Seattle-Tacoma).

In [24]:
airports = data.airports.url
airports

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/airports.csv'

Finalmente, vamos usar uma base de dados para as rotas dos voôs, que contem a origem e destino de cada voô (demarcados pelos campos `origin` e `destination`, repectivamente) com os códigos IATA do aeroporto correspondente:

In [25]:
flights = data.flights_airport.url
flights

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/flights-airport.csv'

Vamos começar criando uma mapa base usando a projeção `albersUsa`, depois adicionando uma camada que plota marcas de círculo para cada aeroporto:

In [26]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(airports).mark_circle(size=9).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip='iata:N'
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

São muitos aeroportos! Obviamente, nem todos eles são os grandes centros de fluxo.

Semelhante ao nosso conjunto de dados de códigos postais, nossos dados dos aeroportos incluem pontos que ficam fora da parte continental dos Estados Unido. Então, vemos novamente pontos no canto superior esquerdo. Podemos querer filtrar esses pontos, mas para isso, primeiro precisaremos saber mais sobre a natureza eles.

*Atualize a projeção do mapa acima para *`albers`* — evitando o comportamento idiossincrático do *`albersUsa`* — para que as localizações reais desses pontos adicionais sejam reveladas!*

Agora, em vez de mostrarmos todos os aeroportos de forma indiferente, vamos identificar os principais centros das rotas considerando o número total de rotas que se originam em cada aeroporto. Usaremos o conjunto de dados `routes` como nossa fonte de dados primária: ele contém uma lista de rotas de vôo que podemos combinar para contar o número de rotas para cada aeroporto de origem (no campo `origin`).

No entanto, o conjunto de dados `routes` não inclui as localizações dos aeroportos! Para conectarmos os dados do `routes` com as localizações, precisamos de um novo tipo de transformação de dados: o `lookup`. A transformação `lookup` pega um valor e um campo em um conjunto de dados primário e o usa como uma chave para procurar informações relacionadas em outra tabela. Neste caso, queremos comparar o código do aeroporto de origem em nosso conjunto de dados `routes` com o campo `iata` do conjunto de dados `airports` e, em seguida, extrair os campos de `latitude` e `longitude` correspondentes.

In [27]:
alt.layer(
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending')
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.LayerChart(...)

*Qual aeroporto dos Estados Unidos tem o maior número de rotas saindo dele?*

Agora que podemos ver os aeroports, gostaríamos de interagir melhor com eles entendendo a estrutura de network do trafégo aéreo. Podemos adicionar a marca gráfica `rule` para representar rotas saindo dos aeroportos de origem até os aeroportos de destino, o que necessita de duas transformações do `lookup` e recuperar as coordenadas para cada ponto desse segmento de reta. Além disso, podemos usar a seleção `single` para filtrar essas rotas tal que apenas as rotas tendo a origem no aeroporto atualmente selecionado seja mostrado.

*Começando do mapa estático acima, você poderia implementar uma versão interativa? Seja livre para pular o código abaixo para se engajar com o mapa interativo primeiro, e pense sobre como você mesmo faria!*

In [ ]:
# Seleção interativa do aeroporto mais próximo
# Seleciona o aeroporto mais próximo do cursor
origin = alt.selection_single(
    on='mouseover', nearest=True,
    fields=['origin'], empty='none'
)

# referência de dados compartilhados para transformações de pesquisa
foreign = alt.LookupData(data=airports, key='iata',
                         fields=['latitude', 'longitude'])

alt.layer(
    # Mapa base dos Estados Unidos
    alt.Chart(alt.topo_feature(usa, 'states')).mark_geoshape(
        fill='#ddd', stroke='#fff', strokeWidth=1
    ),
    # linhas de rota do aeroporto de origem selecionado para os aeroportos de destino
    alt.Chart(flights).mark_rule(
        color='#000', opacity=0.35
    ).transform_filter(
        origin # filtrar apenas para a origem selecionada
    ).transform_lookup(
        lookup='origin', from_=foreign # latitude e longitude da origem
    ).transform_lookup(
        lookup='destination', from_=foreign, as_=['lat2', 'lon2'] # dest lat/lon
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        latitude2='lat2',
        longitude2='lon2',
    ),
    # tamanho aeroportos por número de rotas de saída
    # 1. conjunto de dados de voos-aeroportos agregados
    # 2. pesquisa de dados de localização do conjunto de dados de aeroportos
    # 3. remove Porto Rico (PR) e Ilhas Virgens (VI)
    alt.Chart(flights).mark_circle().transform_aggregate(
        groupby=['origin'],
        routes='count()'
    ).transform_lookup(
        lookup='origin',
        from_=alt.LookupData(data=airports, key='iata',
                             fields=['state', 'latitude', 'longitude'])
    ).transform_filter(
        'datum.state !== "PR" && datum.state !== "VI"'
    ).add_selection(
        origin
    ).encode(
        latitude='latitude:Q',
        longitude='longitude:Q',
        tooltip=['origin:N', 'routes:Q'],
        size=alt.Size('routes:Q', scale=alt.Scale(range=[0, 1000]), legend=None),
        order=alt.Order('routes:Q', sort='descending') # Coloca os circulos menores acima
    )
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

C:\Users\gabyd\AppData\Local\Temp\ipykernel_2112\1801665560.py:3: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use selection_point instead.
  origin = alt.selection_single(
C:\Users\gabyd\AppData\Local\Temp\ipykernel_2112\1801665560.py:45: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(


alt.LayerChart(...)

Passe o mouse sobre o mapa para explorar a rede de voos!

## 7.5 Mapa Coroplético
---

Um [mapa coroplético](https://pt.wikipedia.org/wiki/Mapa_coropl%C3%A9tico) usa regiões sombreadas ou texturizadas para visualizar valores de dados. Mapas com símbolos dimensionados costumam ser mais precisos para leitura, pois as pessoas tendem a estimar melhor as diferenças proporcionais entre as áreas de círculos do que entre tonalidades de cores. No entanto, os mapas coropléticos são populares na prática e particularmente úteis quando muitos símbolos tornam-se perceptivamente excessivos.

Por exemplo, embora os Estados Unidos tem apenas 50 estados, há milhares de condados dentro desses estados. Vamos construir um mapa coroplético da taxa de desemprego por condado, no ano da recessão de 2008. Em alguns casos, os arquivos de entrada GeoJSON ou TopoJSON podem incluir dados estatísticos que podemos visualizar diretamente. Neste caso, no entanto, temos dois arquivos: nosso arquivo TopoJSON, que inclui os limites dos condados (`usa`), e um arquivo de texto separado que contém as estatísticas de desemprego.

In [29]:
unemp = data.unemployment.url
unemp

'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/unemployment.tsv'

Para integrar nossas fontes de dados, precisaremos novamente usar a transformação de busca (`lookup`), enriquecendo nossos dados geoespaciais (`geoshape`) baseados em TopoJSON com as taxas de desemprego. Em seguida, podemos criar um mapa que inclua uma codificação de cores (`color`) para o campo de taxa (`rate`) recuperado.

In [30]:
alt.Chart(alt.topo_feature(usa, 'counties')).mark_geoshape(
    stroke='#aaa', strokeWidth=0.25
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).encode(
    alt.Color('rate:Q',
              scale=alt.Scale(domain=[0, 0.3], clamp=True),
              legend=alt.Legend(format='%')),
    alt.Tooltip('rate:Q', format='.0%')
).project(
    type='albersUsa'
).properties(
    width=900,
    height=500
).configure_view(
    stroke=None
)

alt.Chart(...)

*Examine as taxas de desemprego por condado. Valores mais altos em Michigan podem estar relacionados à indústria automotiva. Condados nas [Grandes Planícies](https://pt.wikipedia.org/wiki/Grandes_Plan%C3%ADcies) e nos estados das Montanhas apresentam tanto taxas baixas **quanto** altas. Essa variação é significativa ou possivelmente um [artefato de tamanhos amostrais menores](https://medium.com/@uwdata/surprise-maps-showing-the-unexpected-e92b67398865)? Para explorar mais, tente alterar o limite superior da escala (por exemplo, para `0.2`) para ajustar o mapeamento de cores.*

Uma preocupação central nos mapas coropléticos é a escolha das cores. Acima, usamos o esquema padrão `yellowgreenblue` do Altair para mapas de calor. Abaixo, comparamos outros esquemas, incluindo um esquema *sequencial de tom único* (`teals`), que varia apenas em luminância; um esquema *sequencial multi-matiz* (`viridis`), que varia tanto em luminância quanto em matiz; e um esquema *divergente* (`blueorange`), que usa um ponto médio branco.

In [31]:
# Função utilitária para gerar uma especificação de mapa para um esquema de cores fornecido.
def map_(scheme):
    return alt.Chart().mark_geoshape().project(type='albersUsa').encode(
        alt.Color('rate:Q', scale=alt.Scale(scheme=scheme), legend=None)
    ).properties(width=305, height=200)

alt.hconcat(
    map_('tealblues'), map_('viridis'), map_('blueorange'),
    data=alt.topo_feature(usa, 'counties')
).transform_lookup(
    lookup='id', from_=alt.LookupData(data=unemp, key='id', fields=['rate'])
).configure_view(
    stroke=None
).resolve_scale(
    color='independent'
)

alt.HConcatChart(...)

*Quais esquemas de cores você considera mais eficaz? Por que isso pode acontecer? Modifique os mapas acima para usar outros esquemas disponíveis, conforme descrito na [documentação dos Esquemas de Cores do Vega](https://vega.github.io/vega/docs/schemes/).*

## 7.6 Projeções Cartográficas
---

Agora que temos alguma experiência na criação de mapas, vamos analisar mais de perto as projeções cartográficas. Conforme explicado pela [Wikipédia](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_cartogr%C3%A1fica).


> <font color="gray"> *Todas as projeções cartográficas necessariamente distorcem a superfície de alguma forma. Dependendo do propósito do mapa, algumas distorções são aceitáveis e outras não são; portanto, diferentes projeções cartográficas existem para preservar algumas propriedades do corpo esférico em detrimento de outras propriedades.*

Algumas das propriedades que podemos querer considerar incluem:

* *Área*: A projeção distorce o tamanho das regiões?
* *Rumo*: Uma linha reta corresponde a uma direção de viagem constante?
* *Distância*: Linhas de comprimento igual correspondem a distâncias iguais no globo?
* *Forma*: A projeção preserva as relações espaciais (ângulos) entre os pontos?

A seleção de uma projeção apropriada depende do caso de uso do mapa. Por exemplo, se estivermos analisando o uso da terra e a extensão das áreas for relevante, podemos escolher uma projeção que preserve a área. Se quisermos visualizar ondas de choque emanando de um terremoto, podemos centralizar o mapa no epicentro do tremor e preservar as distâncias a partir desse ponto. Já se o objetivo for auxiliar a navegação, a preservação do rumo e da forma pode ser mais importante.

Também podemos caracterizar as projeções com base na *superfície de projeção*. As projeções cilíndricas, por exemplo, projetam os pontos da superfície da esfera em um cilindro circundante; ao "desenrolar" o cilindro obtemos o nosso mapa. Como descreveremos a seguir, podemos alternativamente projetar sobre a superfície de um cone (projeções cônicas) ou diretamente em um plano plano (projeções azimutais).

*Vamos primeiro desenvolver nossa intuição interagindo com uma variedade de projeções! [Abra o notebook online de Projeções Cartográficas do Vega-Lite](https://observablehq.com/@vega/vega-lite-cartographic-projections). Use os controles nessa página para selecionar uma projeção e explorar seus parâmetros, como `scale` (escala ou zoom) e translação x/y (panorâmica). Os controles de rotação ([eixo lateral, longitudinal e vertical](https://pt.wikipedia.org/wiki/Eixos_do_avi%C3%A3o)) determinam a orientação do globo em relação à superfície sobre a qual está sendo projetado.*

### Um Tour pelos Tipos Específicos de Projeções

As [projeções cilíndricas](https://jjallaire.github.io/visualization-curriculum/altair_cartographic.html) mapeiam a esfera para um cilindro circundante e, em seguida, "desenrolam" o cilindro. Se o eixo principal do cilindro estiver orientado no sentido norte-sul, os meridianos são mapeados como linhas retas. As projeções [pseudo-cilíndricas](https://en.wikipedia.org/wiki/Map_projection#Pseudocylindrical) representam um meridiano central como uma linha reta, com outros meridianos "curvando-se" para longe do centro.

In [32]:
minimap = map.properties(width=225, height=225)
alt.hconcat(
    minimap.project(type='equirectangular').properties(title='equirectangular'),
    minimap.project(type='mercator').properties(title='mercator'),
    minimap.project(type='transverseMercator').properties(title='transverseMercator'),
    minimap.project(type='naturalEarth1').properties(title='naturalEarth1')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

* [Equirectangular](https://en.wikipedia.org/wiki/Equirectangular_projection) (`equirectangular`): Escala os valores das coordenadas de latitude (`lat`) e longitude diretamente (`lon`).
* [Mercator](https://en.wikipedia.org/wiki/Mercator_projection) (`mercator`): Projeta sobre um cilindro, utilizando a longitude (`lon`) diretamente, mas submetendo a latitude (`lat`) a uma transformação não linear. Linhas retas preservam rumos constantes ([linhas de rhumb](https://en.wikipedia.org/wiki/Rhumb_line)), tornando essa projeção bem adequada para navegação. No entanto, áreas no extremo norte ou sul podem ser grandemente distorcidas.
* [Transversa de Mercator](https://en.wikipedia.org/wiki/Transverse_Mercator_projection) (`transverseMercator`): Uma projeção Mercator, mas com o cilindro de contorno rotacionado para um eixo transversal. Enquanto a projeção Mercator padrão tem maior precisão ao longo do equador, a projeção Transversa de Mercator é mais precisa ao longo do meridiano central.
* [Terra Natural](https://en.wikipedia.org/wiki/Natural_Earth_projection) (`naturalEarth1`): Uma projeção pseudo-cilíndrica projetada para mostrar o mundo todo em uma única visualização.

As [projeções cônicas](https://en.wikipedia.org/wiki/Map_projection#Conic) mapeiam a esfera para um cone e, em seguida, "desenrolam" o cone para o plano. As projeções cônicas são configuradas por dois *paralelos padrão*, que determinam onde o cone intersecta o globo.

In [33]:
minimap = map.properties(width=180, height=130)
alt.hconcat(
    minimap.project(type='conicEqualArea').properties(title='conicEqualArea'),
    minimap.project(type='conicEquidistant').properties(title='conicEquidistant'),
    minimap.project(type='conicConformal', scale=35, translate=[90,65]).properties(title='conicConformal'),
    minimap.project(type='albers').properties(title='albers'),
    minimap.project(type='albersUsa').properties(title='albersUsa')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

* [Cônica de Área Preservada](https://en.wikipedia.org/wiki/Albers_projection) (`conicEqualArea`): Projeção cônica que preserva a área. A forma e a distância não são preservadas, mas a precisão é razoável dentro dos paralelos padrão.

* [Cônica Equidistante](https://en.wikipedia.org/wiki/Equidistant_conic_projection) (`conicEquidistant`): Projeção cônica que preserva a distância ao longo dos meridianos e paralelos padrão.

* [Cônica Conforme](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection) (`conicConformal`): Projeção cônica que preserva a forma (ângulos locais), mas não a área ou a distância.

* [Albers](https://en.wikipedia.org/wiki/Albers_projection) (`albers`): Uma variante da projeção cônica de área preservada, com paralelos padrão otimizados para criar mapas dos Estados Unidos.

* [Albers EUA](https://en.wikipedia.org/wiki/Albers_projection) (`albersUsa`): Uma projeção híbrida para os 50 estados dos Estados Unidos da América. Esta projeção une três projeções Albers com parâmetros diferentes para os EUA continentais, Alasca e Havai.

As [**projeções azimutais**](https://en.wikipedia.org/wiki/Map_projection#Azimuthal_%28projections_onto_a_plane%29) mapeiam a esfera diretamente em um plano.

In [34]:
minimap = map.properties(width=180, height=180)
alt.hconcat(
    minimap.project(type='azimuthalEqualArea').properties(title='azimuthalEqualArea'),
    minimap.project(type='azimuthalEquidistant').properties(title='azimuthalEquidistant'),
    minimap.project(type='orthographic').properties(title='orthographic'),
    minimap.project(type='stereographic').properties(title='stereographic'),
    minimap.project(type='gnomonic').properties(title='gnomonic')
).properties(spacing=10).configure_view(stroke=None)

alt.HConcatChart(...)

* [Azimutal de Área Preservada](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection) (`azimuthalEqualArea`): Projeta com precisão a área em todas as partes do globo, mas não preserva a forma (ângulos locais).

* [Azimutal Equidistante](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection) (`azimuthalEquidistant`): Preserva a distância proporcional do centro da projeção para todos os outros pontos no globo.

* [Ortográfica](https://en.wikipedia.org/wiki/Orthographic_map_projection) (`orthographic`): Projeta um hemisfério visível para um plano distante. Aproximadamente corresponde a uma visão da Terra do espaço exterior.

* [Estereográfica](https://pt.wikipedia.org/wiki/Proje%C3%A7%C3%A3o_estereogr%C3%A1fica) (`stereographic`): Preserva a forma, mas não a área ou a distância.

* [Gnomômica](https://en.wikipedia.org/wiki/Gnomonic_projection) (`gnomonic`): Projeta a superfície da esfera diretamente para um plano tangente. [Círculos máximos](https://en.wikipedia.org/wiki/Great_circle) ao redor da Terra são projetados como linhas retas, mostrando o caminho mais curto entre os pontos.

## 7.8 Tratamento de Dados Geográficos
---

Os exemplos acima são todos retirados da coleção vega-datasets, incluindo dados geométricos (TopoJSON) e tabulares (aeroportos, taxas de desemprego). Um desafio comum para começar com visualização geográfica é coletar os dados necessários para sua tarefa. Existem vários provedores de dados, incluindo serviços como o [United States Geological Survey](https://www.usgs.gov/products/data-and-tools/data-and-tools-topics) e o [U.S. Census Bureau](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).

Em muitos casos, você pode já ter dados existentes com um componente geográfico, mas precisar de medidas ou geometria adicionais. Para ajudar você a começar, aqui está um fluxo de trabalho:

1. Visite [Natural Earth Data](http://www.naturalearthdata.com/downloads/) e navegue para selecionar dados para regiões e resoluções de seu interesse. Baixe o(s) arquivo(s) zip correspondentes.
2. Vá para [MapShaper](https://mapshaper.org/) e arraste seu arquivo zip baixado para a página. Revise os dados conforme desejado e depois "Exporte" os arquivos TopoJSON ou GeoJSON gerados.
3. Carregue os dados exportados do MapShaper para usar com o Altair!

Claro, muitas outras ferramentas &ndash; tanto de código aberto quanto proprietárias &ndash; existem para trabalhar com dados geográficos. Para mais informações sobre tratamento de dados geográficos e criação de mapas, veja a série de tutoriais de Mike Bostock sobre [Command-Line Cartography](https://medium.com/@mbostock/command-line-cartography-part-1-897aa8f8ca2c).

## 7.9 Resumo
---

Neste ponto, nós apenas começamos a explorar o universo da criação de mapas.(Você não esperava que um único capítulo transmitisse séculos de aprendizado, né?) Por exemplo, deixamos de lado tópicos como [_cartogramas_](https://en.wikipedia.org/wiki/Cartogram) e a representação de [_topografia_](https://en.wikipedia.org/wiki/Topography) &mdash; como no livro esclarecedor de Imhof, [_Cartographic Relief Presentation_](https://books.google.com/books?id=cVy1Ms43fFYC). No entanto, agora você deve estar bem equipado para criar uma rica variedade de geo-visualizações. Para mais informações, o livro de MacEachren [_How Maps Work: Representation, Visualization, and Design_](https://books.google.com/books?id=xhAvN3B0CkUC) oferece uma visão valiosa sobre a criação de mapas sob a perspectiva da visualização de dados.
